**Fugu**
- Video streaming is the predominant Interet application, and a key question in video streaming is adaptive bitrate selection (ABR), which decides the compression level selected for each chunk/segment of the video. 
- Fugu: a data-driven ABR algorithm that is basead on model predictive control, but replaces its throughput predictor with a deep NN trained using supervised learning on data recorded in situ (in place), meaning from Fugu's actual deployment environment, Puffer.  
- Objective function: For each chunk, Fugu has a selection of versions of this chunk to choose from 
- Once Fugu decides which chunk to send, we know 2 portionso of the QoE: video quality and video quality variation. Fugu uses a trained NN transmission-time predictor (TTP) 
- For training the TTP, sample from real data collected by any scheme running on Puffer and feed individual user streams to the TTP as training input. Retrain the TTP every day. 

Puffer: https://puffer.stanford.edu 

Puffer is a Stanford University Research study about using ML to improve video-streaming applications, which are used by services such as Youtube, Netflix, ad Twitch. It aims to figure out how to teach a computer to design new algorithms to reduce glitches/stalls in streaming video, improve picture quality, and predict how the capacity of an internet connection will change over time. 

Data Description: https://puffer.stanford.edu/data-description/


In [ ]:
import pandas as pd 
# load in all the data from the data folder 

Task 1: Convert SSIM (structural similarity index measure, which is a method for predicting the percieved video quality) to decibels (dB) using this formula: -10 * np.log10(1 - video_sent['ssim_index'])

Task 2: Group by session ID, then plot the CDF of average SSIM per session

Task 3: Plot the CDF of SSIM of the first chunk (first entry of every session)

Task 4: Plot the Standard deviation of SSIM vs the standard deviation of delivery-rate per session, as a scatter plot

Task 5: Plot the CDF of the average, max, and min delivery rate of each session in the same plot 

Task 6: Show the number of rebuffing per session (Hint: look at the Puffer data description)

Task 7: Plot the average delivery rate vs average SSIM, as a scatter plot. 